In [ ]:
KAIUS EVANS
In this notebook, I go in-depth on the relationship between state mortality rate and state spending on hospitals (and other health 
services). state_data.zip contains data sets on state mortality rate, state spending, state education level and state per capita 
income for the years 1993 through 2015. The data come from different sources such as the US Census, the Bureau of Economic Analysis 
and the US Mortality Database. More specifically, the data files are listed below:

mortality_data.csv
income_data.csv
education data for the years 1993 through 2006 are in the folder education, one file per year: education_1993.csv, education_1994.csv,
..., education_2006.csv
education data for years 2007 through 2015, in one file: education_0715.csv
expenditures data for the years 1993 through 2015 are in the folder expenditure, one file per year: expnd_1993.csv, expnd_1994.csv, 
..., expnd_2015.csv

In [19]:
import pandas as pd
import os

In [ ]:

IMPORT AND CLEAN DATA


In [20]:
import pandas as pd
mort_file_path = '/Users/kaiusevans/Downloads/state_data/mortality_data.csv'
mort_data = pd.read_csv(mort_file_path)
mort_data = mort_data[(mort_data['year'] >= 1993) & (mort_data['year'] <= 2015)]

In [21]:
new_column_names = ['mort_rate', 'prob_death', 'ave_length_surv', 'num_of_surv', 
                    'num_of_deaths', 'num_years_lived', 'num_years_left', 'life_expec']
mort_data.columns = list(mort_data.columns[:3]) + new_column_names + list(mort_data.columns[11:])

print("Renamed mort_data preview:")
print(mort_data.head())

Renamed mort_data preview:
      state  year     age  mort_rate  prob_death  ave_length_surv  \
816  Alaska  1993       0    0.00858     0.00851             0.07   
817  Alaska  1993   4-Jan    0.00060     0.00239             1.82   
818  Alaska  1993   9-May    0.00026     0.00128             2.16   
819  Alaska  1993  14-Oct    0.00056     0.00281             2.75   
820  Alaska  1993   15-19    0.00110     0.00546             2.34   

     num_of_surv  num_of_deaths  num_years_lived  num_years_left  life_expec  
816       100000            851            99210         7553715       75.54  
817        99149            237           396077         7454505       75.19  
818        98912            127           494198         7058427       71.36  
819        98785            277           493303         6564229       66.45  
820        98508            538           491108         6070926       61.63  


In [22]:
mort_data['Age2'] = mort_data['age'].str.split('-').str[0]
mort_data['Age2'] = mort_data['Age2'].replace('110+', '110')
mort_data['Age2'] = pd.to_numeric(mort_data['Age2'])

In [23]:
mort_data['age_group'] = pd.cut(mort_data['Age2'], 
                                bins=[0, 18, 64, mort_data['Age2'].max()], 
                                labels=['<=18', '19-64', '>64'], 
                                include_lowest=True)

print("mort_data with age_group column:")
print(mort_data[['Age2', 'age_group']].head())

mort_data with age_group column:
     Age2 age_group
816     0      <=18
817     4      <=18
818     9      <=18
819    14      <=18
820    15      <=18


In [24]:
mort_data = mort_data.drop(['age', 'Age2'], axis=1)
mort_data = mort_data[['state', 'year', 'age_group', 'mort_rate', 'prob_death', 
                       'ave_length_surv', 'num_of_surv', 'num_of_deaths', 
                       'num_years_lived', 'num_years_left', 'life_expec']]

In [25]:
mort_data.head()

,state,year,age_group,mort_rate,prob_death,ave_length_surv,num_of_surv,num_of_deaths,num_years_lived,num_years_left,life_expec
816,Alaska,1993,<=18,0.00858,0.00851,0.07,100000,851,99210,7553715,75.54
817,Alaska,1993,<=18,0.00060,0.00239,1.82,99149,237,396077,7454505,75.19
818,Alaska,1993,<=18,0.00026,0.00128,2.16,98912,127,494198,7058427,71.36
819,Alaska,1993,<=18,0.00056,0.00281,2.75,98785,277,493303,6564229,66.45
820,Alaska,1993,<=18,0.00110,0.00546,2.34,98508,538,491108,6070926,61.63


In [26]:
mort_data = mort_data.dropna(subset=['mort_rate', 'prob_death', 'ave_length_surv', 
                                     'num_of_surv', 'num_of_deaths', 'num_years_lived', 
                                     'num_years_left', 'life_expec'])
mort_data = mort_data.groupby(['state', 'year', 'age_group'], as_index=False, observed=True).agg({
    'mort_rate': 'mean', 'prob_death': 'mean', 'ave_length_surv': 'mean',
    'num_of_surv': 'mean', 'num_of_deaths': 'mean', 'num_years_lived': 'mean',
    'num_years_left': 'mean', 'life_expec': 'mean'
})
mort_data.head()
#print(f"Row count after Task 6: {len(mort_data)}")

,state,year,age_group,mort_rate,prob_death,ave_length_surv,num_of_surv,num_of_deaths,num_years_lived,num_years_left,life_expec
0,\tMississippi,1993,<=18,0.002146,0.003722,1.908000,99124.000000,370.000000,395172.200000,6.904279e+06,69.636000
1,\tMississippi,1993,19-64,0.004670,0.022883,2.635556,94071.777778,2070.666667,465507.000000,3.623537e+06,38.076667
2,\tMississippi,1993,>64,0.248711,0.560443,2.186000,31075.300000,7951.500000,135242.500000,3.626938e+05,6.860000
3,\tMississippi,1994,<=18,0.002068,0.003616,1.962000,99187.000000,359.600000,395509.000000,6.909326e+06,69.642000
4,\tMississippi,1994,19-64,0.004732,0.023184,2.600000,93931.888889,2095.222222,464718.666667,3.629259e+06,38.180000


In [27]:
mort_data['state'] = mort_data['state'].replace('\tMississippi', 'Mississippi')
mort_data = mort_data.sort_values(by=['state', 'year', 'age_group'])
#mort_data = mort_data.rename(columns={'state': 'State', 'year': 'Year'})
mort_data.head()

,state,year,age_group,mort_rate,prob_death,ave_length_surv,num_of_surv,num_of_deaths,num_years_lived,num_years_left,life_expec
69,Alabama,1993,<=18,0.002596,0.004282,1.960000,98921.800000,425.200000,394146.600000,6.770446e+06,68.422000
70,Alabama,1993,19-64,0.005451,0.026641,2.607778,92887.555556,2368.333333,458790.666667,3.509639e+06,37.254444
71,Alabama,1993,>64,0.231059,0.554784,2.221000,29521.900000,7655.600000,128266.700000,3.427327e+05,6.907000
72,Alabama,1994,<=18,0.002548,0.004234,1.990000,98965.000000,420.200000,394367.200000,6.781783e+06,68.504000
73,Alabama,1994,19-64,0.005362,0.026229,2.631111,92916.000000,2335.222222,459053.888889,3.519781e+06,37.355556


In [28]:
inc_file_path = '/Users/kaiusevans/Downloads/state_data/income_data.csv'
inc_data = pd.read_csv(inc_file_path)
inc_data = pd.wide_to_long(inc_data, stubnames=['pinc'], i=['state'], j='year', sep='.')
inc_data = inc_data.reset_index()
inc_data = inc_data.rename(columns={'pinc': 'income'})
print("inc_data after wide_to_long and reset:")
inc_data.head()
#print("Row count:", len(inc_data))
#print("Unique states:", inc_data['state'].unique())
#print("Unique years:", inc_data['year'].unique())

inc_data after wide_to_long and reset:
        state  year  income
0     Alabama  1993   18129
1      Alaska  1993   25036
2     Arizona  1993   18950
3    Arkansas  1993   16956
4  California  1993   23013
Row count: 1150
Unique states: ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee' 'Texas' 'Utah'
 'Vermont' 'Virginia' 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming']
Unique years: [1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006
 2007 2008 2009 2010 2011 2012 2013 2014 2015]


In [29]:
inc_data = inc_data.sort_values(by=['state', 'year'])
print("inc_data after sort:")
print(inc_data.head())
#print("Row count:", len(inc_data))

inc_data after sort:
       state  year  income
0    Alabama  1993   18129
50   Alabama  1994   18977
100  Alabama  1995   19892
150  Alabama  1996   20630
200  Alabama  1997   21516


In [34]:
educ_dfs = []
for year in range(1993, 2007):
    file_path = f'/Users/kaiusevans/Downloads/state_data/education/education_{year}.csv'
    df = pd.read_csv(file_path)
    df.columns = ['state','year','phs','pcoll']
    educ_dfs.append(df)
educ_0715_path = '/Users/kaiusevans/Downloads/state_data/education_0715.csv'
educ_0715 = pd.read_csv(educ_0715_path)
educ_0715.columns = ['state','year','phs','pcoll']
educ_dfs.append(educ_0715)
educ_data = pd.concat(educ_dfs, ignore_index=True)
#educ_data.columns = list(educ_data.columns[:2]) + ['phs', 'pcoll'] + list(educ_data.columns[4:])
#print("educ_data after concat:")
educ_data.head()
#print("Row count:", len(educ_data))
#print("Unique states:", educ_data['state'].unique())
#print("Unique years:", educ_data['year'].unique())

,state,year,phs,pcoll
0,Alabama,1993,76.0,14.6
1,Alaska,1993,89.2,23.1
2,Arizona,1993,83.9,22.4
3,Arkansas,1993,75.0,15.8
4,California,1993,79.7,25.0


In [36]:
expnd_dfs = []
for year in range(1993, 2016):
    file_path = f'/Users/kaiusevans/Downloads/state_data/expenditure/expnd_{year}.csv'
    df = pd.read_csv(file_path)
    df.columns = ["state", "year", "tot_revenue",
                   "taxes", "tot_expnd", "education", 
                   "public_welfare", "hospital", "health"]
    expnd_dfs.append(df)
expnd_data = pd.concat(expnd_dfs, ignore_index=True)
expnd_data = expnd_data.reset_index(drop=True)
expnd_data = expnd_data.sort_values(by=['state', 'year'])
numeric_cols = expnd_data.columns.drop(['state', 'year'])
expnd_data[numeric_cols] = expnd_data[numeric_cols].replace(",", "", regex = True).astype('int')
expnd_data.head()

,state,year,tot_revenue,taxes,tot_expnd,education,public_welfare,hospital,health
0,Alabama,1993,11389335,4639784,10242374,1920765,2006829,887835,395901
50,Alabama,1994,11599362,4767108,10815221,2101540,2157831,823194,470960
100,Alabama,1995,12448670,5077827,11634629,2260473,2282961,811433,474424
150,Alabama,1996,12741148,5257771,12126587,2240613,2325418,815698,500625
200,Alabama,1997,14007883,5484161,12944867,5175279,2537627,882613,566651


In [ ]:

MERGE DATA


In [37]:
data = pd.merge(inc_data, educ_data, on=['state', 'year'], how='left')
#data = data.rename(columns={'state': 'State', 'year': 'Year'})
data.head()

,state,year,income,phs,pcoll
0,Alabama,1993,18129,76.0,14.6
1,Alabama,1994,18977,72.5,15.2
2,Alabama,1995,19892,74.4,17.3
3,Alabama,1996,20630,75.7,18.0
4,Alabama,1997,21516,77.6,19.3


In [38]:
data = pd.merge(data, expnd_data, on=['state', 'year'], how='left')
data.head()

,state,year,income,phs,pcoll,tot_revenue,taxes,tot_expnd,education,public_welfare,hospital,health
0,Alabama,1993,18129,76.0,14.6,11389335,4639784,10242374,1920765,2006829,887835,395901
1,Alabama,1994,18977,72.5,15.2,11599362,4767108,10815221,2101540,2157831,823194,470960
2,Alabama,1995,19892,74.4,17.3,12448670,5077827,11634629,2260473,2282961,811433,474424
3,Alabama,1996,20630,75.7,18.0,12741148,5257771,12126587,2240613,2325418,815698,500625
4,Alabama,1997,21516,77.6,19.3,14007883,5484161,12944867,5175279,2537627,882613,566651


In [39]:
data = pd.merge(mort_data, data, on=['state', 'year'], how='left')
#print("After Task 14 - head:")
#print(data[['State', 'Year', 'mort_rate', 'income', 'Health', 'Hospitals']].head())
#print("Row count:", len(data))
#print("Non-NaN counts:", data[['mort_rate', 'income', 'Health', 'Hospitals']].count())
data.head()

,state,year,age_group,mort_rate,prob_death,ave_length_surv,num_of_surv,num_of_deaths,num_years_lived,num_years_left,...,income,phs,pcoll,tot_revenue,taxes,tot_expnd,education,public_welfare,hospital,health
0,Alabama,1993,<=18,0.002596,0.004282,1.960000,98921.800000,425.200000,394146.600000,6.770446e+06,...,18129,76.0,14.6,11389335,4639784,10242374,1920765,2006829,887835,395901
1,Alabama,1993,19-64,0.005451,0.026641,2.607778,92887.555556,2368.333333,458790.666667,3.509639e+06,...,18129,76.0,14.6,11389335,4639784,10242374,1920765,2006829,887835,395901
2,Alabama,1993,>64,0.231059,0.554784,2.221000,29521.900000,7655.600000,128266.700000,3.427327e+05,...,18129,76.0,14.6,11389335,4639784,10242374,1920765,2006829,887835,395901
3,Alabama,1994,<=18,0.002548,0.004234,1.990000,98965.000000,420.200000,394367.200000,6.781783e+06,...,18977,72.5,15.2,11599362,4767108,10815221,2101540,2157831,823194,470960
4,Alabama,1994,19-64,0.005362,0.026229,2.631111,92916.000000,2335.222222,459053.888889,3.519781e+06,...,18977,72.5,15.2,11599362,4767108,10815221,2101540,2157831,823194,470960


In [40]:
for df in ['mort_data', 'inc_data', 'educ_data', 'expnd_data']:
    if df in globals():
        del globals()[df]

In [41]:
data.dtypes

state                object
year                  int64
age_group          category
mort_rate           float64
prob_death          float64
ave_length_surv     float64
num_of_surv         float64
num_of_deaths       float64
num_years_lived     float64
num_years_left      float64
life_expec          float64
income                int64
phs                 float64
pcoll               float64
tot_revenue           int64
taxes                 int64
tot_expnd             int64
education             int64
public_welfare        int64
hospital              int64
health                int64
dtype: object

In [42]:
columns_to_convert = ['income', 'tot_revenue', 'taxes', 'tot_expnd', 
                      'education', 'public_welfare', 'hospital', 'health']
#print("Raw data before conversion:")
#print(data[columns_to_convert].head())
data[columns_to_convert] = data[columns_to_convert] / 1e6
#print("\ndata with converted columns (in millions):")
#print(data[columns_to_convert].head())

In [43]:
data['mort_rate'] = data['mort_rate'] * 1e4

#print("data with mort_rate in per 10,000 people:")
#print(data[['mort_rate']].head())

In [44]:
data.describe()

#print("Descriptive statistics for data:")
#print(desc_stats)

,year,mort_rate,prob_death,ave_length_surv,num_of_surv,num_of_deaths,num_years_lived,num_years_left,life_expec,income,phs,pcoll,tot_revenue,taxes,tot_expnd,education,public_welfare,hospital,health
count,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3.450000e+03,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000,3450.000000
mean,2004.000000,850.369552,0.188843,2.258525,76358.355388,3436.387433,340001.412351,3.796730e+06,39.660407,0.033697,63.046991,49.945957,29.855742,12.394309,28.209375,7.260411,6.614074,0.910547,0.873743
std,6.634211,1165.185758,0.251916,0.276008,29557.125216,3489.169291,136269.306107,2.742141e+06,26.380110,0.009615,27.217306,31.997913,39.064751,16.175374,36.756593,9.790643,9.483504,1.274318,1.347735
min,1993.000000,9.400000,0.001390,1.556000,28493.000000,138.600000,123333.900000,3.256397e+05,6.440000,0.015667,18.750000,11.400000,1.942160,0.589069,1.686426,0.201385,0.208404,0.000000,0.024448
25%,1998.000000,19.040000,0.003070,2.014000,36743.250000,305.400000,162413.650000,4.620922e+05,7.522000,0.025967,30.920000,23.100000,8.518436,3.578068,7.816481,1.779178,1.614703,0.104270,0.227264
50%,2004.000000,37.927778,0.018675,2.195500,95143.055556,1723.444444,396310.600000,3.840729e+06,39.946667,0.032911,80.000000,28.650000,17.990004,7.531746,17.015149,4.219294,3.656187,0.509387,0.468598
75%,2010.000000,2423.755000,0.534573,2.601111,99220.600000,8088.300000,467561.305556,6.998749e+06,70.512000,0.040047,86.200000,88.500000,34.804817,15.315386,32.708720,8.938020,7.825282,1.180426,0.977094
max,2015.000000,2937.140000,0.586497,2.750000,99621.800000,8808.700000,480757.222222,7.545226e+06,75.768000,0.068288,93.000000,96.090000,353.385935,151.234165,330.210756,90.276519,109.031702,11.468050,11.943187


In [ ]:

REGRESSIONS


In [ ]:
import statsmodels.api as sm
import numpy as np

In [61]:
data_19_64 = data[data['age_group'] == '19-64'].copy()
data_19_64['log_health'] = np.log(data_19_64['health'])
data_19_64['log_hospital'] = np.log(data_19_64['hospital'])
data_19_64['log_pinc'] = np.log(data_19_64['income'])
data_19_64.dtypes

/opt/miniconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


state                object
year                  int64
age_group          category
mort_rate           float64
prob_death          float64
ave_length_surv     float64
num_of_surv         float64
num_of_deaths       float64
num_years_lived     float64
num_years_left      float64
life_expec          float64
income              float64
phs                 float64
pcoll               float64
tot_revenue         float64
taxes               float64
tot_expnd           float64
education           float64
public_welfare      float64
hospital            float64
health              float64
log_health          float64
log_hospital        float64
log_pinc            float64
dtype: object

In [78]:
data_19_64 = data_19_64.replace([np.inf, -np.inf], np.nan)
data_19_64 = data_19_64.dropna(subset=['mort_rate', 'health', 'hospital', 'income', 'phs', 'pcoll'])
data_19_64 = data_19_64[(data_19_64['health'] > 0) & (data_19_64['hospital'] > 0) & (data_19_64['income'] > 0)]
data_19_64['log_health'] = np.log(data_19_64['health'])
data_19_64['log_hospital'] = np.log(data_19_64['hospital'])
data_19_64['log_income'] = np.log(data_19_64['income'])
X = pd.get_dummies(data_19_64[['log_health', 'log_hospital', 'log_income', 'phs', 'pcoll', 'state']], columns=['state'], drop_first=True)

X = X.astype(float)

tmp1 = smf.ols(formula='mort_rate ~ log_health + log_hospital + log_income + phs + pcoll + C(state)', data=data_19_64).fit()
print(tmp1.summary())

                            OLS Regression Results                            
Dep. Variable:              mort_rate   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     298.5
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:35:11   Log-Likelihood:                -2296.2
No. Observations:                1149   AIC:                             4702.
Df Residuals:                    1094   BIC:                             4980.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [67]:
X = data_19_64[['log_health', 'log_hospital', 'log_pinc', 'phs', 'pcoll']]
X = pd.concat([X, pd.get_dummies(data_19_64['state'], drop_first=True).astype('int')], axis=1)
X = sm.add_constant(X)
X.head()

,const,log_health,log_hospital,log_pinc,phs,pcoll,Alaska,Arizona,Arkansas,California,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
1,1.0,-0.926591,-0.118969,-4.010242,76.0,14.6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,-0.752982,-0.194563,-3.964528,72.5,15.2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.0,-0.745654,-0.208953,-3.917438,74.4,17.3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1.0,-0.691898,-0.203711,-3.881009,75.7,18.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,1.0,-0.568012,-0.124868,-3.838958,77.6,19.3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
y = data_19_64['mort_rate']
spec_2 = sm.OLS(y, X, missing='drop').fit()

#print("Regression results (spec_2) with state dummies for age group '19-64':")
print(spec_2.summary())

                            OLS Regression Results                            
Dep. Variable:              mort_rate   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     298.5
Date:                Thu, 13 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:20:54   Log-Likelihood:                -2296.2
No. Observations:                1149   AIC:                             4702.
Df Residuals:                    1094   BIC:                             4980.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             47.2644      3.990     11.

In [73]:
import statsmodels.api as sm
import numpy as np

data_19_64 = data[data['age_group'] == '19-64'].copy()
data_19_64 = data_19_64[(data_19_64['health'] > 0) & (data_19_64['hospital'] > 0) & (data_19_64['income'] > 0)]
data_19_64['log_health'] = np.log(data_19_64['health'])
data_19_64['log_hospital'] = np.log(data_19_64['hospital'])
data_19_64['log_income'] = np.log(data_19_64['income'])

X = data_19_64[['log_health', 'log_hospital', 'log_income', 'phs', 'pcoll']]
X = pd.concat([X, pd.get_dummies(data_19_64['state'], drop_first=True), 
               pd.get_dummies(data_19_64['year'], drop_first=True)], axis=1)
X = sm.add_constant(X)
y = data_19_64['mort_rate']

spec_3 = sm.OLS(y, X, missing='drop').fit()

print("Regression results (spec_3) with state")
print(spec_3.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [75]:
!pip install stargazer
from stargazer.stargazer import Stargazer

stargazer_table = Stargazer([spec_1, spec_2, spec_3])
stargazer_table.title("Regression Results for Mortality Rate (Age Group 19-64)")
stargazer_table.custom_columns(["spec 1", "spec 2", "spec 3"], [1, 1, 1])
stargazer_table.show_model_numbers(True)

print("Regression table:")
print(stargazer_table.render_ascii())

NameError: name 'spec_1' is not defined